In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping


SEQ_LENGTH = 10  
NUM_FEATURES = 8  
MAX_NUM = 50
EPOCHS = 300  
FUTURE_STEPS = 3  

def create_sequences(data):
    X, y = [], []
    for i in range(len(data) - SEQ_LENGTH):
        X.append(data[i:i + SEQ_LENGTH])
        y.append(data[i + SEQ_LENGTH])
    return np.array(X), np.array(y)

def build_model():
    model = Sequential([
        LSTM(128, return_sequences=True, input_shape=(SEQ_LENGTH, NUM_FEATURES)),  
        LSTM(128),
        Dense(128, activation='relu'),
        Dense(NUM_FEATURES, activation='linear')  
    ])
    model.compile(optimizer='adam', loss='mae')
    return model

def main():
  
    df = pd.read_csv('lottery_results.csv')
    data = df.iloc[:, 1:].values  
    
   
    data_normalized = data / MAX_NUM
    
    
    X, y = create_sequences(data_normalized)
    
   
    model = build_model()
    model.fit(X, y,
              epochs=EPOCHS,
              batch_size=16,
              callbacks=[EarlyStopping(patience=30)],
              verbose=0)
    model.save("modelmax.h5")
    
  
 
    last_sequence = X[-1]  
    future_predictions = []

    for _ in range(FUTURE_STEPS):
        pred = model.predict(last_sequence.reshape(1, SEQ_LENGTH, NUM_FEATURES))
        pred_denormalized = np.round(pred.reshape(NUM_FEATURES) * MAX_NUM).astype(int)

     
        perturbation = np.random.randint(-3, 4, size=NUM_FEATURES)  
        pred_denormalized += perturbation
        pred_denormalized = np.clip(pred_denormalized, 1, MAX_NUM)  

        future_predictions.append(pred_denormalized)

        
        last_sequence = np.vstack([last_sequence[1:], pred / MAX_NUM])

    
    print("\nFuture Predicted Draws:")
    print("========================")
    print(f"⭐ Best Option:\n   {future_predictions[0]}")
    print(f"✅ Better Option:\n   {future_predictions[1]}")
    print(f"🔹 Good Option:\n   {future_predictions[2]}")

if __name__ == "__main__":
    main()


c:\Users\k\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
c:\Users\k\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\callbacks\early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss
  current = self.get_monitor_value(logs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 754ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step

Future Predicted Draws:
⭐ Best Option:
   [ 1  1 13 22 29 35 44 43]
✅ Better Option:
   [ 2  6 10 18 31 42 38 45]
🔹 Good Option:
   [ 4 14 23 27 30 46 50 49]
